ETF DASHBOARD V.2.0

# | Algo DataGenerator
- Fresh Rolling Train DataGenerator
- *inspired by paper "Algorithmic Financial Trading with Deep Convolutional Neural Networks: Time Series to Image Conversion Approach"
- implemenetation reference for neural network prediction model: https://towardsdatascience.com/stock-market-action-prediction-with-convnet-8689238feae3

In [2]:
# Author: Ken Lee 2020.01.12
# Our Library
import ETFFundamentalData as finData
import ETFHistoryDownload as hist
import ETFStockAnalytics as analytic
import ETFPerformanceForecast as perf
# Common Library
import sqlalchemy as sql
import hvplot.pandas
import numpy as np
from pandas import ExcelWriter
import pandas as pd
#import datetime
import time
import os
from pathlib import Path
import logging
from dateutil.relativedelta import relativedelta
from datetime import datetime
import os
import sys
import time

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, LeakyReLU
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger, Callback
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras.initializers import RandomUniform, RandomNormal
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
from tensorflow.keras.utils import get_custom_objects
from sklearn.utils.class_weight import compute_class_weight
import warnings
warnings.filterwarnings('ignore')
from data_generator import DataGenerator, plt
from logger import Logger
#from secrets import api_key
from utils import download_save, seconds_to_minutes
# DB Connection
eft_data_connection_string = 'sqlite:///./Resources/etf.db'
our_portfolio_export = Path('./Resources/our_portfolio.csv')
etf_data_engine = sql.create_engine(eft_data_connection_string, echo=True)
# IF REQUIRED TO RELOAD - SET TRUE
RELOAD_DATA = False

## STEP1. LOAD OUR PORTFOLIO

In [3]:
# Load Data and Create Target Symbol List
sql_query = """
SELECT distinct symbol as symbol
FROM 'OUR_PORTFOLIO_LIST'
UNION
SELECT 'SPY'
"""
our_portfolio_list = pd.read_sql_query(sql_query, eft_data_connection_string)
our_portfolio_list = our_portfolio_list.set_index('symbol')  
symbol_list = []
for symbol in our_portfolio_list.index:
    symbol_list.append(symbol)
print(symbol_list)    

['ACN', 'AMAT', 'AMD', 'APPS', 'ARWR', 'CBRE', 'DE', 'DHR', 'DOCU', 'EL', 'EPAM', 'FB', 'GE', 'GOOG', 'GOOGL', 'GS', 'IRT', 'KEYS', 'KLAC', 'LCII', 'LIN', 'LRCX', 'LSCC', 'LULU', 'MPWR', 'MRVL', 'MSFT', 'MU', 'NEO', 'NXPI', 'PAYC', 'PLD', 'POOL', 'QCOM', 'QQQ', 'QRVO', 'RH', 'ROKU', 'SAIA', 'SEDG', 'SNBR', 'SNPS', 'SPY', 'STX', 'SWKS', 'TER', 'TGT', 'TSLA', 'UPS', 'VICR', 'WST', 'XLK', 'YETI']


In [4]:
# Download PX History if RELOAD_DATA == True
if RELOAD_DATA == True:
    start_date = datetime(2017, 4, 1)
    end_date = datetime(2017, 12, 31)
    hist.fetch_hitorical_allPx(symbol_list, start_date, end_date)
    start_date = datetime(2018, 1, 1)
    end_date = datetime(2018, 12, 31)
    hist.fetch_hitorical_allPx(symbol_list, start_date, end_date)
    start_date = datetime(2019, 1, 1)
    end_date = datetime(2019, 12, 31)
    hist.fetch_hitorical_allPx(symbol_list, start_date, end_date)
    start_date = datetime(2020, 1, 1)
    end_date = datetime(2020, 12, 31)
    hist.fetch_hitorical_allPx(symbol_list, start_date, end_date)
    start_date = datetime(2021, 1, 1)
    end_date = datetime(2021, 12, 31)
    hist.fetch_hitorical_allPx(symbol_list, start_date, end_date)
    start_date = datetime(2022, 1, 1)
    end_date = datetime(2022, 12, 31)
    hist.fetch_hitorical_allPx(symbol_list, start_date, end_date)

In [8]:
# Create fresh_rolling_train data for ML From PX_HISTORY data 
# by Calculating RSI, WilliamR, MFI, CMF, CMO, SMA, EMA, WMA, HMA, TRIX, CCI, DPO, KST, DMI, BB Band, KDJK, RSV, EMO, volumeDelta
if RELOAD_DATA == True:
    for symbol in symbol_list:
        config = tf.compat.v1.ConfigProto()
        config.gpu_options.allow_growth = True
        session = tf.compat.v1.Session(config=config)
        print("running stock_cnn.py", symbol)
        pd.options.display.width = 0
        company_code = symbol
        strategy_type = 'original'
        ROOT_PATH = "."
        iter_changes = "fresh_rolling_train"  # label for changes in this run iteration
        INPUT_PATH = os.path.join(ROOT_PATH, "stock_history", company_code)
        OUTPUT_PATH = os.path.join(ROOT_PATH, "outputs", iter_changes)
        LOG_PATH = OUTPUT_PATH + os.sep + "logs"
        LOG_FILE_NAME_PREFIX = "log_{}_{}_{}".format(company_code, strategy_type, iter_changes)
        PATH_TO_STOCK_HISTORY_DATA = os.path.join(ROOT_PATH, "stock_history")

        if not os.path.exists(INPUT_PATH):
            os.makedirs(INPUT_PATH)
            print("Input Directory created", INPUT_PATH)

        if not os.path.exists(OUTPUT_PATH):
            os.makedirs(OUTPUT_PATH)
            print("Output Directory created", OUTPUT_PATH)

        BASE_URL = ""  # api key from alpha vantage service
        data_file_name = company_code + ".csv"
        PATH_TO_COMPANY_DATA = os.path.join(PATH_TO_STOCK_HISTORY_DATA, company_code, data_file_name)
        print(INPUT_PATH)
        print(OUTPUT_PATH)
        print(PATH_TO_COMPANY_DATA)

        logger = Logger(LOG_PATH, LOG_FILE_NAME_PREFIX)

        start_time = time.time()
        data_gen = DataGenerator(company_code, PATH_TO_COMPANY_DATA, OUTPUT_PATH, strategy_type, False, logger)
        


running stock_cnn.py ACN
.\stock_history\ACN
.\outputs\fresh_rolling_train
.\stock_history\ACN\ACN.csv
1 ) 17-01-2022 10_03_36 MainThread INFO\ Initialized logging at path ./outputs/fresh_rolling_train/logs/log_ACN_original_fresh_rolling_train_17-01-2022_10_03_36.log
path to company data: .\stock_history\ACN\ACN.csv
2 ) 17-01-2022 10_03_36 MainThread DEBUG\ Data for ACN ready to use
3 ) 17-01-2022 10_03_36 MainThread DEBUG\ Technical indicators already calculated. Loading...
4 ) 17-01-2022 10_03_37 MainThread DEBUG\ Dropped 0 nan rows before label calculation
labels already calculated
5 ) 17-01-2022 10_03_37 MainThread DEBUG\ Number of Technical indicator columns for train/test are 443
6 ) 17-01-2022 10_03_50 MainThread DEBUG\ common selected featues:292, ['kdjk_26', 'roc_26', 'rsi_14', 'bb_13', 'wr_14', 'kst_14', 'wr_7', 'roc_24', 'wr_12', 'cci_22', 'cmf_19', 'trix_17', 'kdjk_21', 'roc_19', 'wr_22', 'dpo_6', 'roc_8', 'rsi_6', 'hma_7', 'dpo_7', 'dpo_15', 'dpo_22', 'kst_11', 'kdjk_12', 

  0%|          | 0/21 [00:00<?, ?it/s]

Calculation of RSI Done 2.0 minutes 15.0 seconds
Calculating WilliamR


  0%|          | 0/21 [00:00<?, ?it/s]

Calculating MFI


...

Calculation of EMA Done 0.0 minutes 1.0 seconds
Calculating EOM


  0%|          | 0/21 [00:00<?, ?it/s]

Calculation of EOM done 0.0 minutes 0.0 seconds
Calculating volume delta
Calculation of Volume Delta done 0.0 minutes 0.0 seconds
5 ) 17-01-2022 14_52_19 MainThread DEBUG\ Saving dataframe...
6 ) 17-01-2022 14_52_22 MainThread DEBUG\ Dropped 439 nan rows before label calculation
7 ) 17-01-2022 14_52_22 MainThread DEBUG\ creating label with original paper strategy
Calculating labels


  0%|          | 0/766 [00:00<?, ?it/s]

window_middle 5
window_middle 6
window_middle 7
window_middle 8
window_middle 9
window_middle 10
window_middle 11
window_middle 12
window_middle 13
window_middle 14
window_middle 15
window_middle 16
window_middle 17
window_middle 18
window_middle 19
window_middle 20
window_middle 21
window_middle 22
window_middle 23
window_middle 24
window_middle 25
window_middle 26
window_middle 27
window_middle 28
window_middle 29
window_middle 30
window_middle 31
window_middle 32
window_middle 33
window_middle 34
window_middle 35
window_middle 36
window_middle 37
window_middle 38
window_middle 39
window_middle 40
window_middle 41
window_middle 42
window_middle 43
window_middle 44
window_middle 45
window_middle 46
window_middle 47
window_middle 48
window_middle 49
window_middle 50
window_middle 51
window_middle 52
window_middle 53
window_middle 54
window_middle 55
window_middle 56
window_middle 57
window_middle 58
window_middle 59
window_middle 60
window_middle 61
window_middle 62
window_middle 63
wi